In [108]:
!pip install -q aif360
import numpy as np
import pandas as pd
import aif360 as aif
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.impute import SimpleImputer
from aif360.sklearn import metrics
from scipy.stats import chisquare
from scipy.stats import f_oneway
import requests
import json
from time import sleep

In [70]:
resumes = pd.read_csv('ResumeLinks.csv')

In [71]:
resumes

,Link
0,https://drive.usercontent.google.com/download?...
1,https://drive.usercontent.google.com/download?...
2,https://drive.usercontent.google.com/download?...
3,https://drive.usercontent.google.com/download?...
4,https://drive.usercontent.google.com/download?...
5,https://drive.usercontent.google.com/download?...
6,https://drive.usercontent.google.com/download?...
7,https://drive.usercontent.google.com/download?...
8,https://drive.usercontent.google.com/download?...
9,https://drive.usercontent.google.com/download?...


In [72]:
url = 'https://dev.api.talentmarx.in/api/v1/ml/extract-text' # Extract text api

In [73]:
for index, row in resumes.iterrows():
    data = {"url" : row['Link']}
    response = requests.post(url, json=data)
    text_from_response = response.text
    resumes.at[index, 'text'] = text_from_response
    if index == 5:
        sleep(5)

In [74]:
resumes

,Link,text
0,https://drive.usercontent.google.com/download?...,"""MYTHILI MURUGESAN Email : mmythilinov19@gmai..."
1,https://drive.usercontent.google.com/download?...,""" Neelesh Kumar Singh Phone: (+91) 9555388069 ..."
2,https://drive.usercontent.google.com/download?...,"""PRIYANKA 2 Years experience in UI development..."
3,https://drive.usercontent.google.com/download?...,"""Priyanshu Tiwari B.Tech, Computer Science tiw..."
4,https://drive.usercontent.google.com/download?...,"""Vardaraj.D Mobile: +91-9886133212 Religion: H..."
5,https://drive.usercontent.google.com/download?...,""" Mr. Ramu Vinod Vishwakarma Age: 34 Religion..."
6,https://drive.usercontent.google.com/download?...,"""Sagar Namdev Khairnar Age: 37 Religion: Hindu..."
7,https://drive.usercontent.google.com/download?...,""" Name : SAKTHIVADIVEL GANESAN Position : Seni..."
8,https://drive.usercontent.google.com/download?...,"""SHRUTI AGRAWAL Front End UI Developer Age: 3..."
9,https://drive.usercontent.google.com/download?...,""" Curriculum Vitae Tanmay Kundu UI DEVELOPER A..."


In [75]:
job_description = '''Seeking a talented UI/UX Designer to spearhead our mobile app’s design, creating intuitive and engaging user experiences.Work alongside product managers and engineers in a collaborative environment to translate our vision into a functional and visually appealing app.

Responsibilities:
• 5+ years of experience as a UI/UX Designer with hands on experience in Crafting wireframes, user stories, journeys, and mockups for a diverse range of devices.
• Solve design challenges with elegant solutions.
• Employ a user-centered approach to design, with a cycle of testing and iteration.
• Engage in collaborative problem-solving and idea generation.

What We Offer:
• A competitive salary with performance incentives.
• The flexibility of remote work.
• The chance to contribute significantly to the educational sector.
• A vibrant culture that fosters professional growth and innovation.

Engagement Type:
• Payroll and Compliacne to be managed by: sigiq.ai
• The client will share the agreement with the talents directly to manage their Payroll
• Client / Talent Agreement - Full-time Permanent position

This Role Requires:

Must-Have Skills:
• Demonstrated proficiency in UI/UX design tools (Adobe XD, Sketch, Figma, InVision) with a strong portfolio showcasing mobile app designs.
• In-depth knowledge of responsive web and mobile app design, including iOS and Android guidelines.
• Expertise in UX best practices, mobile-first and responsive design.
• Exceptional visual design skills focused on typography, layout, and color.
• Effective teamwork skills, with the ability to turn abstract ideas into concrete design outcomes.
• Experience in aligning detailed design efforts with user needs and product direction.

Nice-to-Have Skills:
• Familiarity with HTML, CSS, JavaScript for design implementation.

How to apply for this opportunity?
• Register or login on our portal & fill out the application form.
• Clear the given AI Screening (30 min) and Click on 'Apply' to get shortlisted
• Once it's all done, your profile will be shared with the client for the Interview round.
• When selected, just meet the client and get your exciting career started!

About our Client - sigiq.ai:

About Uplers:

Our goal is to make hiring reliable, simple, and fast. Our role will be to help all our talents find and apply for relevant contractual onsite opportunities and progress in their career. We will support any grievances or challenges you may face during the engagement. You will also be assigned to a dedicated Talent Success Coach during the engagement.

(Note: There are many more opportunities apart from this on the portal. Depending on the assessments you clear, you can apply for them as well).

So, if you are ready for a new challenge, a great work environment, and an opportunity to take your career to the next level, don't hesitate to apply today. We are waiting for you'''

In [76]:
url = 'https://dev.api.talentmarx.in/api/v1/ml/similarity/' #score api

In [84]:
text_from_response['detail'][0].keys()

dict_keys(['type', 'loc', 'msg', 'input', 'url'])

In [85]:
for index, row in resumes.iterrows():
    data = {
      "queryDocumentString": job_description,
      "documentStrings": [
        row['text']
      ]
    }
    response = requests.post(url, json=data)
    text_from_response = eval(response.text)
    resumes.at[index, 'score'] = text_from_response['similarities']
    if index == 5:
        sleep(5)

In [86]:
resumes.head()

,Link,text,score
0,https://drive.usercontent.google.com/download?...,"""MYTHILI MURUGESAN Email : mmythilinov19@gmai...",0.557548
1,https://drive.usercontent.google.com/download?...,""" Neelesh Kumar Singh Phone: (+91) 9555388069 ...",0.714022
2,https://drive.usercontent.google.com/download?...,"""PRIYANKA 2 Years experience in UI development...",0.510026
3,https://drive.usercontent.google.com/download?...,"""Priyanshu Tiwari B.Tech, Computer Science tiw...",0.577816
4,https://drive.usercontent.google.com/download?...,"""Vardaraj.D Mobile: +91-9886133212 Religion: H...",0.527642


In [89]:
url = 'https://dev.rp.talentmarx.in/post_json' #data extraction api

In [123]:
from datetime import date 
todays_date = date.today() 
current_year = todays_date.year

for index, row in resumes.iterrows():
    data = {"url" : row['Link']}
    response = requests.post(url, json=data)
    text_from_response = eval(response.text)
    genders = text_from_response['ResumeParserData']['Gender']
    dob = text_from_response['ResumeParserData']['DateOfBirth'][:4]
    age = current_year - int(dob if dob != '' else current_year)
    resumes.at[index, 'gender'] = gender
    resumes.at[index, 'age'] = age if age != 0 else pd.NA
    if index == 5:
        sleep(5)

In [131]:
resumes = resumes.sort_values(by="score", ascending=False)
resumes.head()

,Link,text,score,gender,age
1,https://drive.usercontent.google.com/download?...,""" Neelesh Kumar Singh Phone: (+91) 9555388069 ...",0.714022,Male,34.0
6,https://drive.usercontent.google.com/download?...,"""Sagar Namdev Khairnar Age: 37 Religion: Hindu...",0.668283,Male,NaN
8,https://drive.usercontent.google.com/download?...,"""SHRUTI AGRAWAL Front End UI Developer Age: 3...",0.639950,Male,38.0
7,https://drive.usercontent.google.com/download?...,""" Name : SAKTHIVADIVEL GANESAN Position : Seni...",0.582533,Male,40.0
5,https://drive.usercontent.google.com/download?...,""" Mr. Ramu Vinod Vishwakarma Age: 34 Religion...",0.578646,Male,34.0


In [134]:
max_selection = 4
old_age = 35
for index, _ in resumes.iterrows():
    resumes.at[index, "Selected"] = 1 if index <= max_selection else 0
resumes['age'] = resumes['age'].apply(lambda x: 1 if x > 35 else 0)

In [31]:
selected = np.random.randint(2, size=df.shape[0])

In [32]:
Selected = pd.DataFrame(selected, columns=['Selected'])

In [33]:
df['Selected'] = Selected

In [34]:
df.head()

,Name,Age,Religion,Job Title,Gender,Experience,Selected
0,Akshatha,40.0,Hindu,Business Analytics,F,9.5,0
1,Vinod Kumar S,46.0,Hindu,Business Analytics,M,14.0,1
2,Tushar Kumar Saxena,41.0,Hindu,Business Analytics,M,16.0,0
3,ANUJ MALHOTRA,38.0,Hindu,Business Analytics,M,17.0,0
4,SVM Ajay SriPennada,36.0,Hindu,Business Analytics,M,4.6,0


In [35]:
df.isna().sum()

Name          0
Age           2
Religion      1
Job Title     0
Gender        0
Experience    1
Selected      0
dtype: int64

In [42]:
df.dropna(inplace=True)

In [55]:
metrics.statistical_parity_difference(df['Selected'], prot_attr=df['Gender'], priv_group='M')

0.05714285714285716

In [57]:
metrics.statistical_parity_difference(df['Selected'], prot_attr=df['Religion'], priv_group='Hindu')

-0.13025210084033617

In [58]:
metrics.disparate_impact_ratio(df['Selected'], prot_attr=df['Gender'], priv_group='M')

1.105263157894737

In [59]:
metrics.disparate_impact_ratio(df['Selected'], prot_attr=df['Religion'], priv_group='Hindu')

0.7669172932330827

In [78]:
metrics.class_imbalance(y_true=df['Selected'], prot_attr=df['Gender'], priv_group='M')

-0.8666666666666667

In [77]:
metrics.class_imbalance(y_true=df['Selected'], prot_attr=df['Religion'], priv_group='Hindu')

-0.8133333333333334

In [70]:
metrics.kl_divergence(y_true=df['Selected'], prot_attr=df['Gender'], priv_group='M')

0.006711901983134094

In [62]:
aif.detectors.bias_scan(df.drop(columns=['Selected']), df['Selected'])

({'Name': ['ANUJ MALHOTRA',
   'ASHISH KUMAR KHAMARI',
   'Aabid A.S. Mulani',
   'Akshatha',
   'Anil Kumar',
   'Arunesh Sharma',
   'Charan Kumar Reddy',
   'Jayakkumar Krishnasamy',
   'Madan Mohan',
   'Mudassar L Shaikh\t',
   'NITIN CHANDORKAR',
   'Neelakanta R',
   'PRAHALADHA',
   'SVM Ajay SriPennada',
   'Sabeeh Ashhar ',
   'Sachindra D',
   'Sakthivadivel Ganesan',
   'Sandeep Cheedepudi',
   'Sankalp S. Rao',
   'Sathyandran S K',
   'Saurabh Suhasaria',
   'Thilagavathi Saravanan',
   'Tushar Kumar Saxena'],
  'Age': [32.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 45.0, 46.0],
  'Job Title': ['Android', 'Business Analytics', 'QA Testing', 'Sales', 'UI'],
  'Experience': [4.6,
   6.0,
   9.5,
   11.0,
   12.0,
   12.7,
   13.0,
   14.0,
   15.0,
   16.0,
   17.0,
   18.0,
   19.0,
   20.0,
   21.0],
  'Gender': ['F', 'M'],
  'Religion': ['Hindu', 'Muslim']},
 18.155)

In [80]:
def standard_deviation_ratio(scores, groups):
    std_dev_group_A = np.std(scores[groups[0]])
    std_dev_group_B = np.std(scores[groups[1]])
    return std_dev_group_A / std_dev_group_B

def standard_deviation_disparity(scores, groups):
    std_dev_group_A = np.std(scores[groups[0]])
    std_dev_group_B = np.std(scores[groups[1]])
    std_dev_overall = np.std(scores)
    return (std_dev_group_A - std_dev_group_B) / std_dev_overall

def equal_opportunity(true_positives, total_positives):
    return true_positives / total_positives


def chi2_test(observed, alpha=0.05):
    expected = [sum(observed) / len(observed)] * 6
    chi2, p = chisquare(observed, f_exp=expected)
    return p < alpha

def anova_bias(col1, col2):
    res = f_oneway(col1.tolist(),col2.tolist())
    print("H0 Accepted") if res.pvalue > 0.05 else print("H0 Rejected")
    
def demographic_parity(col1, col2,threshold):
    ratio1 = sum(col1) / len(col1)
    ratio2 = sum(col2) / len(col2)
    print("Bias exists") if(abs(ratio1-ratio2) > threshold) else print("No bias")